In [19]:
import glob
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog


In [20]:
# Color features

def getSpatialFeatures(image, size=(32, 32)):
    '''
    Create the feature vector using cv2.resize().ravel()
    '''
    features = cv2.resize(image, size).ravel() 
    return features

def getHistFeatures(image, numberOfBins=32, binRange=(0, 256)):
    '''
    Compute the histogram of the color channels separately
    '''
    channelHist1 = np.histogram(image[:,:,0], bins=numberOfBins, range=binRange)
    channelHist2 = np.histogram(image[:,:,1], bins=numberOfBins, range=binRange)
    channelHist3 = np.histogram(image[:,:,2], bins=numberOfBins, range=binRange)
    histFeatures = np.concatenate((channelHist1[0], channelHist2[0], channelHist3[0]))
    return histFeatures

def getColorFeatures(image, size=(32, 32), numberOfBins=32, binRange=(0, 256)):
    '''
    Get spatial features and histogram features of colors
    '''
    spatialFeatures = getSpatialFeatures(image, size)
    histFeatures = getHistFeatures(image, numberOfBins, binRange)
    return np.concatenate(spatialFeatures, histFeatures)


In [21]:
# HOG features

def getGradientFeatures(image, orientations, pixelPerCell, cellPerBlock, visualise=False, featureVector=True):
    '''
    Get HOG features
    '''
    # Call with two outputs if vis==True
    if visualise == True:
        features, hogImage = hog(image, orientations=orientations, pixels_per_cell=(pixelPerCell, pixelPerCell), 
                                 cells_per_block=(cellPerBlock, cellPerBlock), transform_sqrt=True, 
                                 visualise=visualise, feature_vector=featureVector)
        return features, hogImage
    # Otherwise call with one output
    else:      
        features = hog(image, orientations=orientations, pixels_per_cell=(pixelPerCell, pixelPerCell),
                       cells_per_block=(cellPerBlock, cellPerBlock), transform_sqrt=True, 
                       visualise=visualise, feature_vector=featureVector)
        return features

    
def getHogFeatures(image, orientations, pixelPerCell, cellPerBlock, visualise, featureVector, hogChannel):
    '''
    Get HOG features for all channels or a single channel
    '''
    if hogChannel == 'ALL':
        hogFeatures = []
        for channel in range(image.shape[2]):
            hogFeatures.append(getGradientFeatures(image[:,:,channel], orientations, 
                                                   pixelPerCell, cellPerBlock, visualise, featureVector))
            hogFeatures = np.ravel(hogFeatures)        
    else:
        hogFeatures = getGradientFeatures(image[:,:,hogChannel], orientations, 
                                          pixelPerCell, cellPerBlock, visualise, featureVector)
        
    return hogFeatures
    

In [22]:
# 

def extractFeatures(images, colorSpace, featureFunction, **featureFunctionArguments):
    '''
    Extract features of the images using the featureFunction
    '''
    features = []
    for file in images:
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if colorSpace != 'RGB':
            if colorSpace == 'HSV':
                featureImage = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif colorSpace == 'LUV':
                featureImage = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif colorSpace == 'HLS':
                featureImage = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif colorSpace == 'YUV':
                featureImage = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif colorSpace == 'YCrCb':
                featureImage = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            featureImage = np.copy(image)
            
        features.append(featureFunction(featureImage, **featureFunctionArguments))
    
    return features